![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [2]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

filename_features = "./dados/features.txt"
filename_labels = "./dados/activity_labels.txt"

filename_subtrain = "./dados/subject_train.txt"
filename_xtrain = "./dados/X_train.txt"
filename_ytrain = "./dados/y_train.txt"

filename_subtest = "./dados/subject_test.txt"
ffilename_xtest = "./dados/X_test.txt"
filename_ytest = "./dados/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\davi_\AppData\Local\Temp\ipykernel_14424\1374599716.py:22: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\davi_\AppData\Local\Temp\ipykernel_14424\1374599716.py:25: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\davi_\AppData\Local\Temp\ipykernel_14424\1374599716.py:29: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

- Pode-se observar uma boa acurácia para as bases de treino e teste, porém um tempo de processamento consideravelmente grande para o treinamento de apenas um modelo. 

- Quase 4 segundos para treinar apenas um modelo de classificação, o que provavemente se dá em conta do grande número de linhas e colunas na base de treino.

In [9]:
%%time

clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(X_train, y_train)


print(f'Acurácia treino = {accuracy_score(y_train, clf.predict(X_train))}')
print(f'Acurácia teste  = {accuracy_score(y_test, clf.predict(X_test))}')

Acurácia treino = 0.9757889009793254
Acurácia teste  = 0.8795385137427892
CPU times: total: 3.17 s
Wall time: 3.43 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

- Ao aplicar o PCA e reduzir a base para apenas uma componente, foi possível reduzir absurdamente o tempo de processamento para treinar o modelo. Porém a acurácia também caiu bastante (tanto com a base de treino quanto com a de teste).

- O que deve se dar ao fato de ter reduzido demais as variáveis do modelo.

In [14]:
%%time

# Aplico o PCA
prcomp = PCA(n_components=1).fit(X_train)

# Gero a componente mais importante pras bases
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Crio o modelo
clf = DecisionTreeClassifier(ccp_alpha=0.001).fit(pc_treino, y_train)

# Avaliação das acurácias
print(f'Acurácia treino = {accuracy_score(y_train, clf.predict(pc_treino))}')
print(f'Acurácia teste  = {accuracy_score(y_test, clf.predict(pc_teste))}')

Acurácia treino = 0.499727965179543
Acurácia teste  = 0.45707499151679676
CPU times: total: 469 ms
Wall time: 117 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


Definição da função pra aplicar o PCA

In [35]:
def aplicar_PCA(xtreino: pd.DataFrame, # variáveis explicativas treino
                xteste: pd.DataFrame,  # variáveis explicativas teste
                ytreino: pd.Series,    # variável resposta treino
                yteste: pd.Series,     # variável resposta teste
                num_componentes: int) -> None:
    
    
    # aplico o PCA com o número de componentes dolicitados
    prcomp = PCA(n_components = num_componentes).fit(xtreino)
    
    
    # gero as variáveis
    pc_treino = prcomp.transform(xtreino)
    pc_teste  = prcomp.transform(xteste)
    
    
    # treino o modelo com as variaveis geradas
    clf = DecisionTreeClassifier(ccp_alpha = 0.001).fit(pc_treino, ytreino)
    
    # print acurácias
    #print(f"Resultados com {num_componentes} componentes")
    print(f"Acurácia treino = { round(clf.score(pc_treino, ytreino), 4) }")
    print(f"Acurácia teste  = { round(clf.score(pc_teste, yteste), 4) }")
    
    
    return None

PCA com 2 componentes

In [36]:
%%time

aplicar_PCA(X_train, X_test, y_train, y_test, num_componentes=2)

Acurácia treino = 0.6128
Acurácia teste  = 0.5847
CPU times: total: 406 ms
Wall time: 114 ms


PCA com 5 componentes

In [37]:
%%time

aplicar_PCA(X_train, X_test, y_train, y_test, num_componentes=5)

Acurácia treino = 0.846
Acurácia teste  = 0.7886
CPU times: total: 531 ms
Wall time: 139 ms


PCA com 10 componentes

In [38]:
%%time

aplicar_PCA(X_train, X_test, y_train, y_test, num_componentes=10)

Acurácia treino = 0.8924
Acurácia teste  = 0.8239
CPU times: total: 766 ms
Wall time: 178 ms


PCA com 50 componentes

In [39]:
%%time

aplicar_PCA(X_train, X_test, y_train, y_test, num_componentes=50)

Acurácia treino = 0.9168
Acurácia teste  = 0.829
CPU times: total: 1.75 s
Wall time: 550 ms


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Aparentemente, aplicar o PCA com 10 componentes teve um ótimo resultado, as acurácias de treino e teste subiram bastante comparadas as de menos componentes, e o tempo de processamento não mudou muita coisa.

No entanto, ao aplicar com 50 componentes, apesar da acurácia ter aumentando um pouco para ambas as bases, o aumento foi muito pequeno e o tempo de processamentou aumento em quase 5x. 

Portanto, pode-se concluir que é mais vantajoso manter com apenas 10 componentes.